In [110]:
#import libraries
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from sklearn.cluster import DBSCAN
from newspaper import Article
from nltk.corpus import stopwords 
from bs4 import BeautifulSoup 
from geopy.geocoders import Nominatim
from itertools import chain
import operator
import string
import warnings 
import folium
from collections import Counter
import spacy 
from spacy.lang.en import English # updated
from spacy.matcher import Matcher
pd.options.mode.chained_assignment = None  # default='warn'
#pd.set_option('display.max_colwidth', 0)
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [111]:
df = pd.read_csv('/home/rangika/Documents/PHD/DATA/NIWA6.csv', encoding='utf8', sep='\t', usecols=['Title','url'])
df.head()
print(df.describe())

                                                    Title  \
count                                                  89   
unique                                                 89   
top     Headline: Auckland cliff collapse: Mud ‘burst ...   
freq                                                    1   

                                                      url  
count                                                  89  
unique                                                 89  
top     https://www.stuff.co.nz/the-press/news/1175037...  
freq                                                    1  


In [112]:
df.head()

,Title,url
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...
1,Headline: Insurance companies face deluge of ...,https://www.stuff.co.nz/timaru-herald/news/117...
2,Headline: Cars damaged in severe Timaru hails...,https://www.stuff.co.nz/timaru-herald/news/117...
3,Headline: Record insurance repairs for cars s...,https://www.odt.co.nz/star-news/star-districts...
4,"""Headline: Heavy snow closes roads in Canterbu...",https://www.rnz.co.nz/news/national/396380/hea...


In [113]:
from nltk.corpus import stopwords 
from bs4 import BeautifulSoup 
import re  
stop_words = set(stopwords.words('english'))
def  text_cleaner(text):
    newString = text
    newString = newString.replace('/', '')
    newString = re.sub('Headline: ', '', newString)# Remove 'Headline:' part
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub('\""(.+?)\""', '', newString)#remove double quotes
    #newString = re.sub(r'[^a-zA-Z0-9]+', '', newString)#remove special characters
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub(r'\b[A-Z]+\b', '', newString)#remove capital letters
    newString = re.sub('"','', newString)
    newString = re.sub(r"http\S+", "", newString)#remove twitter links
    #newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    #newString = re.sub(r"'s\b","",newString)
    newString = re.sub(r'[^\w\s]','',newString)#remove punctuation
    return newString


df['Tidy_Title'] = np.vectorize(text_cleaner)(df['Title'])
df.head()

,Title,url,Tidy_Title
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...
1,Headline: Insurance companies face deluge of ...,https://www.stuff.co.nz/timaru-herald/news/117...,Insurance companies face deluge of hail damage...
2,Headline: Cars damaged in severe Timaru hails...,https://www.stuff.co.nz/timaru-herald/news/117...,Cars damaged in severe Timaru hailstorm failin...
3,Headline: Record insurance repairs for cars s...,https://www.odt.co.nz/star-news/star-districts...,Record insurance repairs for cars smashed by h...
4,"""Headline: Heavy snow closes roads in Canterbu...",https://www.rnz.co.nz/news/national/396380/hea...,Heavy snow closes roads in Canterbury and Otag...


In [114]:
import spacy

nlp = spacy.load("en_core_web_lg")
df['Where_Title'] = df['Tidy_Title'].apply(lambda row: ",".join([ent.text for ent in nlp(row).ents if (ent.label_=='GPE' or ent.label_=='LOC' or ent.label_=='FAC')]))
df.head(1)

,Title,url,Tidy_Title,Where_Title
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru


In [115]:
df['When_Title'] = df['Tidy_Title'].apply(lambda row: ",".join([ent.text for ent in nlp(row).ents if ent.label_== "DATE" or ent.label_=='TIME']))
df.head()

,Title,url,Tidy_Title,Where_Title,When_Title
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru,
1,Headline: Insurance companies face deluge of ...,https://www.stuff.co.nz/timaru-herald/news/117...,Insurance companies face deluge of hail damage...,,
2,Headline: Cars damaged in severe Timaru hails...,https://www.stuff.co.nz/timaru-herald/news/117...,Cars damaged in severe Timaru hailstorm failin...,Timaru,
3,Headline: Record insurance repairs for cars s...,https://www.odt.co.nz/star-news/star-districts...,Record insurance repairs for cars smashed by h...,Timaru,
4,"""Headline: Heavy snow closes roads in Canterbu...",https://www.rnz.co.nz/news/national/396380/hea...,Heavy snow closes roads in Canterbury and Otag...,"Canterbury,Otago",


### Description

In [116]:
from newspaper import Article

eventURLList = df['url'].tolist()

#PubDate = []
#NewsTitle = []
NewsData = []
for url in eventURLList:
    article = Article(url)
    article.download()
    try:
        article.parse()
    #PubDate.append(article.publish_date)
        NewsData.append(article.text)
    except:
        pass
    #NewsTitle.append(article.title)
    


df['Description'] = pd.Series(NewsData, index=df.index)
df.head()

,Title,url,Tidy_Title,Where_Title,When_Title,Description
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru,,A hail and thunder storm left a trail of destr...
1,Headline: Insurance companies face deluge of ...,https://www.stuff.co.nz/timaru-herald/news/117...,Insurance companies face deluge of hail damage...,,,HVS Motors Timaru owner Hamish Gardyne at thei...
2,Headline: Cars damaged in severe Timaru hails...,https://www.stuff.co.nz/timaru-herald/news/117...,Cars damaged in severe Timaru hailstorm failin...,Timaru,,VTNZ's Washdyke branch manager Russell Burt ch...
3,Headline: Record insurance repairs for cars s...,https://www.odt.co.nz/star-news/star-districts...,Record insurance repairs for cars smashed by h...,Timaru,,An Australian-based team of hail damage repair...
4,"""Headline: Heavy snow closes roads in Canterbu...",https://www.rnz.co.nz/news/national/396380/hea...,Heavy snow closes roads in Canterbury and Otag...,"Canterbury,Otago",,Heavy snowfalls in South Canterbury and Otago ...


In [117]:
df = df.dropna(subset=['Description'])
len(df)

89

### Description

In [118]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [119]:
from nltk.corpus import stopwords 
from bs4 import BeautifulSoup 
import re  
stop_words = set(stopwords.words('english'))
def  text_cleaner2(text):
    newString = text
    
    #newString = newString.replace('/', '')
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub('\""(.+?)\""', '', newString)#remove double quotes
    newString = re.sub('Advertisement', '', newString)# Remove 'Headline:' part
    newString = re.sub('''Video will play in Play now Don't auto play''', '', newString)
    newString = re.sub('Never auto play', '', newString)
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = (re.sub(r'\b[A-Z]+\b', '', newString))#remove capital letters
    newString = re.sub('"','', newString)
    newString = re.sub(r"http\S+", "", newString)#remove twitter links
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub(',','', newString)#remove commas
    #newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

df['Tidy_News'] = np.vectorize(text_cleaner2)(df['Description'])
df.head()

,Title,url,Tidy_Title,Where_Title,When_Title,Description,Tidy_News
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru,,A hail and thunder storm left a trail of destr...,hail thunder storm left trail destruction Sout...
1,Headline: Insurance companies face deluge of ...,https://www.stuff.co.nz/timaru-herald/news/117...,Insurance companies face deluge of hail damage...,,,HVS Motors Timaru owner Hamish Gardyne at thei...,Motors Timaru owner Hamish Gardyne storage sit...
2,Headline: Cars damaged in severe Timaru hails...,https://www.stuff.co.nz/timaru-herald/news/117...,Cars damaged in severe Timaru hailstorm failin...,Timaru,,VTNZ's Washdyke branch manager Russell Burt ch...,Washdyke branch manager Russell Burt checks ca...
3,Headline: Record insurance repairs for cars s...,https://www.odt.co.nz/star-news/star-districts...,Record insurance repairs for cars smashed by h...,Timaru,,An Australian-based team of hail damage repair...,Australian-based team hail damage repairers se...
4,"""Headline: Heavy snow closes roads in Canterbu...",https://www.rnz.co.nz/news/national/396380/hea...,Heavy snow closes roads in Canterbury and Otag...,"Canterbury,Otago",,Heavy snowfalls in South Canterbury and Otago ...,Heavy snowfalls South Canterbury Otago closed ...


In [120]:
from spacy.lang.en import English # updated

nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated


def split_sentences(document):
    SentencesList = []
    doc = nlp(document)
    sentences = [sent.string.strip() for sent in doc.sents][:6]
    SentencesList.append(sentences)
    #print(SentencesList)
    return SentencesList



#split_sentences(string1)
#df2['first_sentences'] = np.vectorize(split_sentences)(df2['Description'])

df['first_sentences'] = df['Tidy_News'].apply(split_sentences)
df.head(1)

,Title,url,Tidy_Title,Where_Title,When_Title,Description,Tidy_News,first_sentences
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru,,A hail and thunder storm left a trail of destr...,hail thunder storm left trail destruction Sout...,[[hail thunder storm left trail destruction So...


In [121]:
df['liststring'] = df['first_sentences'].apply(lambda x: ','.join(map(str, x)))
df['liststring'] = df.liststring.astype(str).str.replace('\[|\]|\'', '')
df.head(1)

,Title,url,Tidy_Title,Where_Title,When_Title,Description,Tidy_News,first_sentences,liststring
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru,,A hail and thunder storm left a trail of destr...,hail thunder storm left trail destruction Sout...,[[hail thunder storm left trail destruction So...,hail thunder storm left trail destruction Sout...


In [122]:
import spacy

nlp = spacy.load("en_core_web_lg")
df['Where_Body'] = df['liststring'].apply(lambda row: ",".join([ent.text for ent in nlp(row).ents if (ent.label_=='GPE' or ent.label_=='LOC' or ent.label_=='FAC')]))
df.head()

,Title,url,Tidy_Title,Where_Title,When_Title,Description,Tidy_News,first_sentences,liststring,Where_Body
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru,,A hail and thunder storm left a trail of destr...,hail thunder storm left trail destruction Sout...,[[hail thunder storm left trail destruction So...,hail thunder storm left trail destruction Sout...,"South Canterbury,Timaru,Timaru,Timaru,New Zealand"
1,Headline: Insurance companies face deluge of ...,https://www.stuff.co.nz/timaru-herald/news/117...,Insurance companies face deluge of hail damage...,,,HVS Motors Timaru owner Hamish Gardyne at thei...,Motors Timaru owner Hamish Gardyne storage sit...,[[Motors Timaru owner Hamish Gardyne storage s...,Motors Timaru owner Hamish Gardyne storage sit...,"Timaru,Timaru"
2,Headline: Cars damaged in severe Timaru hails...,https://www.stuff.co.nz/timaru-herald/news/117...,Cars damaged in severe Timaru hailstorm failin...,Timaru,,VTNZ's Washdyke branch manager Russell Burt ch...,Washdyke branch manager Russell Burt checks ca...,[[Washdyke branch manager Russell Burt checks ...,Washdyke branch manager Russell Burt checks ca...,Timaru
3,Headline: Record insurance repairs for cars s...,https://www.odt.co.nz/star-news/star-districts...,Record insurance repairs for cars smashed by h...,Timaru,,An Australian-based team of hail damage repair...,Australian-based team hail damage repairers se...,[[Australian-based team hail damage repairers ...,Australian-based team hail damage repairers se...,"Timaru,Queensland,Timaru,Timaru"
4,"""Headline: Heavy snow closes roads in Canterbu...",https://www.rnz.co.nz/news/national/396380/hea...,Heavy snow closes roads in Canterbury and Otag...,"Canterbury,Otago",,Heavy snowfalls in South Canterbury and Otago ...,Heavy snowfalls South Canterbury Otago closed ...,[[Heavy snowfalls South Canterbury Otago close...,Heavy snowfalls South Canterbury Otago closed ...,"South Canterbury Otago,State Highway Springs,e..."


In [123]:
df['When_Body'] = df['liststring'].apply(lambda row: ",".join([ent.text for ent in nlp(row).ents if ent.label_== "DATE" or ent.label_=='TIME']))
df.head()

,Title,url,Tidy_Title,Where_Title,When_Title,Description,Tidy_News,first_sentences,liststring,Where_Body,When_Body
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru,,A hail and thunder storm left a trail of destr...,hail thunder storm left trail destruction Sout...,[[hail thunder storm left trail destruction So...,hail thunder storm left trail destruction Sout...,"South Canterbury,Timaru,Timaru,Timaru,New Zealand","noon,Wednesday,November 2019,noon"
1,Headline: Insurance companies face deluge of ...,https://www.stuff.co.nz/timaru-herald/news/117...,Insurance companies face deluge of hail damage...,,,HVS Motors Timaru owner Hamish Gardyne at thei...,Motors Timaru owner Hamish Gardyne storage sit...,[[Motors Timaru owner Hamish Gardyne storage s...,Motors Timaru owner Hamish Gardyne storage sit...,"Timaru,Timaru","Wednesday,weeks,mid-week,yesterday"
2,Headline: Cars damaged in severe Timaru hails...,https://www.stuff.co.nz/timaru-herald/news/117...,Cars damaged in severe Timaru hailstorm failin...,Timaru,,VTNZ's Washdyke branch manager Russell Burt ch...,Washdyke branch manager Russell Burt checks ca...,[[Washdyke branch manager Russell Burt checks ...,Washdyke branch manager Russell Burt checks ca...,Timaru,
3,Headline: Record insurance repairs for cars s...,https://www.odt.co.nz/star-news/star-districts...,Record insurance repairs for cars smashed by h...,Timaru,,An Australian-based team of hail damage repair...,Australian-based team hail damage repairers se...,[[Australian-based team hail damage repairers ...,Australian-based team hail damage repairers se...,"Timaru,Queensland,Timaru,Timaru","November,two weeks ago"
4,"""Headline: Heavy snow closes roads in Canterbu...",https://www.rnz.co.nz/news/national/396380/hea...,Heavy snow closes roads in Canterbury and Otag...,"Canterbury,Otago",,Heavy snowfalls in South Canterbury and Otago ...,Heavy snowfalls South Canterbury Otago closed ...,[[Heavy snowfalls South Canterbury Otago close...,Heavy snowfalls South Canterbury Otago closed ...,"South Canterbury Otago,State Highway Springs,e...",afternoon


In [124]:
def time_parse(line):
    timestrings = []
    r = re.compile(r'(\d*[:.]\d{2}\s?(?:AM|PM|am|pm))')
    x = r.findall(line)
    timestrings.append(x)
    return timestrings

In [125]:
df['When_Body_Regex'] = df['liststring'].apply(lambda row: time_parse(row))
df['When_Body_Regex'] = df.When_Body_Regex.astype(str).str.replace('\[|\]|\'', '')
df.head()

,Title,url,Tidy_Title,Where_Title,When_Title,Description,Tidy_News,first_sentences,liststring,Where_Body,When_Body,When_Body_Regex
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru,,A hail and thunder storm left a trail of destr...,hail thunder storm left trail destruction Sout...,[[hail thunder storm left trail destruction So...,hail thunder storm left trail destruction Sout...,"South Canterbury,Timaru,Timaru,Timaru,New Zealand","noon,Wednesday,November 2019,noon",2.40pm
1,Headline: Insurance companies face deluge of ...,https://www.stuff.co.nz/timaru-herald/news/117...,Insurance companies face deluge of hail damage...,,,HVS Motors Timaru owner Hamish Gardyne at thei...,Motors Timaru owner Hamish Gardyne storage sit...,[[Motors Timaru owner Hamish Gardyne storage s...,Motors Timaru owner Hamish Gardyne storage sit...,"Timaru,Timaru","Wednesday,weeks,mid-week,yesterday",
2,Headline: Cars damaged in severe Timaru hails...,https://www.stuff.co.nz/timaru-herald/news/117...,Cars damaged in severe Timaru hailstorm failin...,Timaru,,VTNZ's Washdyke branch manager Russell Burt ch...,Washdyke branch manager Russell Burt checks ca...,[[Washdyke branch manager Russell Burt checks ...,Washdyke branch manager Russell Burt checks ca...,Timaru,,
3,Headline: Record insurance repairs for cars s...,https://www.odt.co.nz/star-news/star-districts...,Record insurance repairs for cars smashed by h...,Timaru,,An Australian-based team of hail damage repair...,Australian-based team hail damage repairers se...,[[Australian-based team hail damage repairers ...,Australian-based team hail damage repairers se...,"Timaru,Queensland,Timaru,Timaru","November,two weeks ago",
4,"""Headline: Heavy snow closes roads in Canterbu...",https://www.rnz.co.nz/news/national/396380/hea...,Heavy snow closes roads in Canterbury and Otag...,"Canterbury,Otago",,Heavy snowfalls in South Canterbury and Otago ...,Heavy snowfalls South Canterbury Otago closed ...,[[Heavy snowfalls South Canterbury Otago close...,Heavy snowfalls South Canterbury Otago closed ...,"South Canterbury Otago,State Highway Springs,e...",afternoon,


In [126]:
bigdata = df.head(1)
bigdata

,Title,url,Tidy_Title,Where_Title,When_Title,Description,Tidy_News,first_sentences,liststring,Where_Body,When_Body,When_Body_Regex
0,Headline: Giant hail stones hammer Timaru as ...,https://www.stuff.co.nz/timaru-herald/news/117...,Giant hail stones hammer Timaru as storm moves...,Timaru,,A hail and thunder storm left a trail of destr...,hail thunder storm left trail destruction Sout...,[[hail thunder storm left trail destruction So...,hail thunder storm left trail destruction Sout...,"South Canterbury,Timaru,Timaru,Timaru,New Zealand","noon,Wednesday,November 2019,noon",2.40pm


In [158]:
WT1 = bigdata['Where_Title'].tolist()
WT2 = bigdata['Where_Body'].tolist()
WT1, WT2

(['Timaru'], ['South Canterbury,Timaru,Timaru,Timaru,New Zealand'])

In [159]:
WT = WT1 + WT2 
WT

words = [w.replace('the', '') for w in WT]
words = [w.replace('Eastern Bay', 'Eastern Bay of Plenty') for w in words]
words = [w.replace('Eastern', '') for w in words]
words = [w.replace('house', '') for w in words]
words = [w.replace('New Zealand', '') for w in words]

str_list = list(filter(None, words))
str_list

['Timaru', 'South Canterbury,Timaru,Timaru,Timaru,']

In [160]:
str_list2 = ",".join(str_list).split(',')
str_list2 = [x.strip(' ') for x in str_list2] #Remove space before ' Bay of Plenty
str_list2

['Timaru', 'South Canterbury', 'Timaru', 'Timaru', 'Timaru', '']

In [161]:
str_list2 = list(filter(None, str_list2))
str_list2

['Timaru', 'South Canterbury', 'Timaru', 'Timaru', 'Timaru']

In [162]:
counts = Counter(str_list2)
counts_where = dict(counts)
print(counts_where)

{'Timaru': 4, 'South Canterbury': 1}


In [163]:
df_locations = pd.DataFrame(list(counts_where.items()), columns=['locations', 'frequency'])
df_locations

,locations,frequency
0,Timaru,4
1,South Canterbury,1


In [164]:
loc_list = df_locations.locations.tolist()
loc_list

['Timaru', 'South Canterbury']

In [165]:
titletext = bigdata[["Tidy_Title"]]
titletext

,Tidy_Title
0,Giant hail stones hammer Timaru as storm moves...


In [166]:
titletext['locs_in_title'] = titletext.Tidy_Title.str.findall(f'({"|".join(loc_list)})', flags=re.IGNORECASE)
titletext

,Tidy_Title,locs_in_title
0,Giant hail stones hammer Timaru as storm moves...,[Timaru]


In [167]:
titletext['locs_in_title'] = titletext.locs_in_title.astype(str).str.replace('\[|\]|\'', '')
titletext

,Tidy_Title,locs_in_title
0,Giant hail stones hammer Timaru as storm moves...,Timaru


In [168]:
titletext['locs_in_title'].replace('', np.nan, inplace=True)
titletext

,Tidy_Title,locs_in_title
0,Giant hail stones hammer Timaru as storm moves...,Timaru


In [169]:
titletext = titletext.dropna(subset=['locs_in_title'])
titletext

,Tidy_Title,locs_in_title
0,Giant hail stones hammer Timaru as storm moves...,Timaru


In [170]:
titletext['title_score'] = 1.0
titletext

,Tidy_Title,locs_in_title,title_score
0,Giant hail stones hammer Timaru as storm moves...,Timaru,1.0


In [171]:
bodytext = bigdata[["liststring"]]

bodytext["Shape"]= bodytext["liststring"].str.split("., ")

bodytext = (bodytext.explode("Shape").reset_index(drop=True))

bodytext

,liststring,Shape
0,hail thunder storm left trail destruction Sout...,hail thunder storm left trail destruction Sout...
1,hail thunder storm left trail destruction Sout...,Timaru hammered hail storm causing damage buil...
2,hail thunder storm left trail destruction Sout...,Large golf ball-sized hail stones began fallin...
3,hail thunder storm left trail destruction Sout...,Damage massive hail stones reported throughout...
4,hail thunder storm left trail destruction Sout...,Hail stones size golf balls hit Timaru lunchti...
5,hail thunder storm left trail destruction Sout...,Fire Emergency New Zealand Mid-South Canterbur...


In [172]:
grouped = bodytext.groupby('liststring', as_index=False)
grouped_index = grouped.apply(lambda x: x.reset_index(drop = True)).reset_index()
result = grouped_index.drop('level_0',axis = 1).set_index('level_1')
result

,liststring,Shape
level_1,,
0,hail thunder storm left trail destruction Sout...,hail thunder storm left trail destruction Sout...
1,hail thunder storm left trail destruction Sout...,Timaru hammered hail storm causing damage buil...
2,hail thunder storm left trail destruction Sout...,Large golf ball-sized hail stones began fallin...
3,hail thunder storm left trail destruction Sout...,Damage massive hail stones reported throughout...
4,hail thunder storm left trail destruction Sout...,Hail stones size golf balls hit Timaru lunchti...
5,hail thunder storm left trail destruction Sout...,Fire Emergency New Zealand Mid-South Canterbur...


In [173]:
result = result.reset_index()
result

,level_1,liststring,Shape
0,0,hail thunder storm left trail destruction Sout...,hail thunder storm left trail destruction Sout...
1,1,hail thunder storm left trail destruction Sout...,Timaru hammered hail storm causing damage buil...
2,2,hail thunder storm left trail destruction Sout...,Large golf ball-sized hail stones began fallin...
3,3,hail thunder storm left trail destruction Sout...,Damage massive hail stones reported throughout...
4,4,hail thunder storm left trail destruction Sout...,Hail stones size golf balls hit Timaru lunchti...
5,5,hail thunder storm left trail destruction Sout...,Fire Emergency New Zealand Mid-South Canterbur...


In [174]:
import re
# result['location_mapped_title'] = (result.Shape.str
#                              .findall('|'.join(loc_list))
#                              .str[0])

#x = result.Shape.str.extractall(f'({"|".join(loc_list)})', flags=re.IGNORECASE)
result['locs_in_lines'] = result.Shape.str.findall(f'({"|".join(loc_list)})', flags=re.IGNORECASE)
result

,level_1,liststring,Shape,locs_in_lines
0,0,hail thunder storm left trail destruction Sout...,hail thunder storm left trail destruction Sout...,[South Canterbury]
1,1,hail thunder storm left trail destruction Sout...,Timaru hammered hail storm causing damage buil...,[Timaru]
2,2,hail thunder storm left trail destruction Sout...,Large golf ball-sized hail stones began fallin...,[]
3,3,hail thunder storm left trail destruction Sout...,Damage massive hail stones reported throughout...,[Timaru]
4,4,hail thunder storm left trail destruction Sout...,Hail stones size golf balls hit Timaru lunchti...,[Timaru]
5,5,hail thunder storm left trail destruction Sout...,Fire Emergency New Zealand Mid-South Canterbur...,[South Canterbury]


In [175]:
result['locs_in_lines'] = result.locs_in_lines.astype(str).str.replace('\[|\]|\'', '')

result

,level_1,liststring,Shape,locs_in_lines
0,0,hail thunder storm left trail destruction Sout...,hail thunder storm left trail destruction Sout...,South Canterbury
1,1,hail thunder storm left trail destruction Sout...,Timaru hammered hail storm causing damage buil...,Timaru
2,2,hail thunder storm left trail destruction Sout...,Large golf ball-sized hail stones began fallin...,
3,3,hail thunder storm left trail destruction Sout...,Damage massive hail stones reported throughout...,Timaru
4,4,hail thunder storm left trail destruction Sout...,Hail stones size golf balls hit Timaru lunchti...,Timaru
5,5,hail thunder storm left trail destruction Sout...,Fire Emergency New Zealand Mid-South Canterbur...,South Canterbury


In [176]:
result['locs_in_lines'].replace("", np.nan, inplace=True)
result = result.dropna(subset=['locs_in_lines'])
result

,level_1,liststring,Shape,locs_in_lines
0,0,hail thunder storm left trail destruction Sout...,hail thunder storm left trail destruction Sout...,South Canterbury
1,1,hail thunder storm left trail destruction Sout...,Timaru hammered hail storm causing damage buil...,Timaru
3,3,hail thunder storm left trail destruction Sout...,Damage massive hail stones reported throughout...,Timaru
4,4,hail thunder storm left trail destruction Sout...,Hail stones size golf balls hit Timaru lunchti...,Timaru
5,5,hail thunder storm left trail destruction Sout...,Fire Emergency New Zealand Mid-South Canterbur...,South Canterbury


In [177]:
def f(row):
    if row['level_1'] == 0:
        val = 1
    elif row['level_1'] == 1:
        val = 0.5
    elif row['level_1'] == 2:
        val = 0.25
    elif row['level_1'] == 3:
        val = 0.125
    elif row['level_1'] == 4:
        val = 0.0625
    else:
        val = 0.03125
    return val

In [178]:
result['body_score'] = result.apply(f, axis=1)
result

,level_1,liststring,Shape,locs_in_lines,body_score
0,0,hail thunder storm left trail destruction Sout...,hail thunder storm left trail destruction Sout...,South Canterbury,1.00000
1,1,hail thunder storm left trail destruction Sout...,Timaru hammered hail storm causing damage buil...,Timaru,0.50000
3,3,hail thunder storm left trail destruction Sout...,Damage massive hail stones reported throughout...,Timaru,0.12500
4,4,hail thunder storm left trail destruction Sout...,Hail stones size golf balls hit Timaru lunchti...,Timaru,0.06250
5,5,hail thunder storm left trail destruction Sout...,Fire Emergency New Zealand Mid-South Canterbur...,South Canterbury,0.03125


In [179]:
#df_calc = result[['locs_in_lines', 'pos_score']]
result["locations"]=result["locs_in_lines"].str.split(", ")
result = result.explode("locations")

#df_calc = df_calc.groupby('splited')['C'].sum().reset_index() 
print(result)

   level_1                                         liststring  \
0        0  hail thunder storm left trail destruction Sout...   
1        1  hail thunder storm left trail destruction Sout...   
3        3  hail thunder storm left trail destruction Sout...   
4        4  hail thunder storm left trail destruction Sout...   
5        5  hail thunder storm left trail destruction Sout...   

                                               Shape     locs_in_lines  \
0  hail thunder storm left trail destruction Sout...  South Canterbury   
1  Timaru hammered hail storm causing damage buil...            Timaru   
3  Damage massive hail stones reported throughout...            Timaru   
4  Hail stones size golf balls hit Timaru lunchti...            Timaru   
5  Fire Emergency New Zealand Mid-South Canterbur...  South Canterbury   

   body_score         locations  
0     1.00000  South Canterbury  
1     0.50000            Timaru  
3     0.12500            Timaru  
4     0.06250            Tim

In [180]:
df_bodytext_score = result.groupby('locations')['body_score'].sum().reset_index() 
df_bodytext_score

,locations,body_score
0,South Canterbury,1.03125
1,Timaru,0.68750


In [181]:
df_title_score = titletext[['locs_in_title', 'title_score']]
df_title_score['locations'] = df_title_score['locs_in_title']
df_title_score

,locs_in_title,title_score,locations
0,Timaru,1.0,Timaru


In [182]:
df_position_score = pd.merge(df_bodytext_score, df_title_score, on=['locations'],how='outer')
df_position_score

,locations,body_score,locs_in_title,title_score
0,South Canterbury,1.03125,NaN,NaN
1,Timaru,0.68750,Timaru,1.0


In [183]:
df_position_score['full_position_score'] = df_position_score['body_score'] + df_position_score['title_score']
df_position_score

,locations,body_score,locs_in_title,title_score,full_position_score
0,South Canterbury,1.03125,NaN,NaN,NaN
1,Timaru,0.68750,Timaru,1.0,1.6875


In [184]:
df_position_score.full_position_score.fillna(df_position_score.body_score, inplace=True)
df_position_score

,locations,body_score,locs_in_title,title_score,full_position_score
0,South Canterbury,1.03125,NaN,NaN,1.03125
1,Timaru,0.68750,Timaru,1.0,1.68750


In [185]:
df_locations

,locations,frequency
0,Timaru,4
1,South Canterbury,1


In [186]:
df_position_and_frequency = pd.merge(df_locations, df_position_score, on=['locations'],how='outer')
df_position_and_frequency

,locations,frequency,body_score,locs_in_title,title_score,full_position_score
0,Timaru,4,0.68750,Timaru,1.0,1.68750
1,South Canterbury,1,1.03125,NaN,NaN,1.03125


In [187]:
df_position_and_frequency['position_and_frequency_score'] = df_position_and_frequency['full_position_score'] + df_position_and_frequency['frequency']
df_position_and_frequency

,locations,frequency,body_score,locs_in_title,title_score,full_position_score,position_and_frequency_score
0,Timaru,4,0.68750,Timaru,1.0,1.68750,5.68750
1,South Canterbury,1,1.03125,NaN,NaN,1.03125,2.03125


In [188]:
from geopy.geocoders import Nominatim

def geo_code(line):
    x = []
    y = []

    line2 = 'New Zealand'
    n = line + ' + ' + line2
    r = '"' + n + '"'
    


    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode(r)
    x.append(location.latitude)
    y.append(location.longitude)
    return x,y

In [189]:
loc_list

['Timaru', 'South Canterbury']

In [190]:
cordinates = [geo_code(item) for item in loc_list]
cordinates

[([-44.3930254], [171.2509786]), ([-43.49417615], [171.8098447584145])]

In [191]:
list_of_coords = [(x[0][0], x[1][0]) for x in cordinates]
list_of_coords

[(-44.3930254, 171.2509786), (-43.49417615, 171.8098447584145)]

In [192]:
from math import hypot
from geopy.distance import geodesic

#https://stackoverflow.com/questions/23086078/how-to-iterate-through-a-list-of-coordinates-and-calculate-distance-between-them
def distance(p1,p2):
    x = geodesic(p1, p2).miles
    """Euclidean distance between two points."""
#     x1,y1 = p1
#     x2,y2 = p2

    return x

from itertools import combinations
#list_of_coords = [(-37.9503756, 176.93828736155422), (-43.6131375, 172.3051281), (-38.1272259, 177.1307049)]



distancecombi = [distance(*combo) for combo in combinations(list_of_coords,2)]
distancecombi

[68.0318602432256]

In [193]:
import itertools

locationombi = list(itertools.combinations(loc_list, 2))# cordinates = list(map(geo_code, listofloc))
locationombi

[('Timaru', 'South Canterbury')]

In [194]:
import pandas as pd
import numpy as np

df_loc = pd.DataFrame(
    {'locpairs': locationombi,
     'dispairs': distancecombi
    })
df_loc

,locpairs,dispairs
0,"(Timaru, South Canterbury)",68.03186


In [195]:
df_loc['locpairs'] = df_loc.locpairs.astype(str).str.replace('\(|\)|\'', '')
df_loc

,locpairs,dispairs
0,"Timaru, South Canterbury",68.03186


In [196]:
def x(row):
    if row['dispairs'] <= 10:
        val = 1
    elif row['dispairs'] <= 20:
        val = 0.5
    elif row['dispairs'] <= 30:
        val = 0.25
    elif row['dispairs'] <= 40:
        val = 0.125
    elif row['dispairs'] <= 50:
        val = 0.0625
    else:
        val = 0.03125
    return val

In [197]:
df_loc['loc_score'] = df_loc.apply(x, axis=1)
df_loc

,locpairs,dispairs,loc_score
0,"Timaru, South Canterbury",68.03186,0.03125


In [200]:
df_relatedness = df_loc.loc[df_loc['loc_score'] >= 0.5]
df_relatedness

,locpairs,dispairs,loc_score


In [203]:
if not df_relatedness.empty:
    df_relatedness["locations"]=df_relatedness["locpairs"].str.split(", ")
    df_relatedness = df_relatedness.explode("locations")
    df_relatedness

In [204]:
df_position_and_frequency

,locations,frequency,body_score,locs_in_title,title_score,full_position_score,position_and_frequency_score
0,Timaru,4,0.68750,Timaru,1.0,1.68750,5.68750
1,South Canterbury,1,1.03125,NaN,NaN,1.03125,2.03125
